In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,0,28,2024-08-06 20:34:20
1,大安區,21,0,21,2024-08-06 20:19:14
2,大安區,16,2,14,2024-08-06 20:34:19
3,大安區,11,4,7,2024-08-06 20:34:20
4,大安區,16,2,14,2024-08-06 20:34:20
...,...,...,...,...,...
1433,臺大公館校區,30,0,30,2024-08-06 20:27:15
1434,臺大公館校區,35,0,35,2024-08-06 20:28:15
1435,臺大公館校區,24,18,6,2024-08-06 20:34:20
1436,臺大公館校區,40,6,34,2024-08-06 20:33:20


In [2]:
bins = [0,1,4,7,11,100]
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']

df['可借'] = pd.cut(df['可借數量'],bins=bins,right=False,labels=labels)
df

,行政區域,總數量,可借數量,可還數量,時間,可借
0,大安區,28,0,28,2024-08-06 20:34:20,0輛
1,大安區,21,0,21,2024-08-06 20:19:14,0輛
2,大安區,16,2,14,2024-08-06 20:34:19,1~3輛
3,大安區,11,4,7,2024-08-06 20:34:20,4~6輛
4,大安區,16,2,14,2024-08-06 20:34:20,1~3輛
...,...,...,...,...,...,...
1433,臺大公館校區,30,0,30,2024-08-06 20:27:15,0輛
1434,臺大公館校區,35,0,35,2024-08-06 20:28:15,0輛
1435,臺大公館校區,24,18,6,2024-08-06 20:34:20,10輛以上
1436,臺大公館校區,40,6,34,2024-08-06 20:33:20,4~6輛


In [9]:
df.groupby(by=['行政區域','可借'])[['可借數量']].count()

/tmp/ipykernel_8553/611826505.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(by=['行政區域','可借'])[['可借數量']].count()


可借數量
行政區域 可借         
中山區  0輛       20
     1~3輛     33
     4~6輛     33
     7~10輛    24
     10輛以上    49
...          ...
萬華區  0輛        9
     1~3輛     10
     4~6輛     13
     7~10輛    16
     10輛以上    32

[65 rows x 1 columns]

In [8]:
df1 = df.groupby(by=['行政區域','可借'])[['可借數量']].count()
df1.rename(columns={'可借數量':'站點數量'})

/tmp/ipykernel_8553/3955084304.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df1 = df.groupby(by=['行政區域','可借'])[['可借數量']].count()


站點數量
行政區域 可借         
中山區  0輛       20
     1~3輛     33
     4~6輛     33
     7~10輛    24
     10輛以上    49
...          ...
萬華區  0輛        9
     1~3輛     10
     4~6輛     13
     7~10輛    16
     10輛以上    32

[65 rows x 1 columns]